# Creating Pandapower Networks

This Tutorial will introduce the user into the pandapower datastructure and how to create networks through the pandapower API. The following minimal example contains the most commont elements that are supported by the Pandapower format. For an example that contains all pandapower elements (3 winding transformers, ward equivalents, impedances), see the [advanced tutorial](create_advanced.ipynb) for network creation.

<img src="pics/example_network_simple.png">

The datastructure of the pandapower framework is based on the python library pandas. A pandapower network consist of a separate element table for each element type that is used in the network. Each element table consists of a column for each parameter and a row for each element. By executing the follwing code cells you generate the various element tables.  You can find detailed descriptions about each parameter in the pandapower documentation under bulletpoint "Datastructures and Elements".

### Empty Network

First, we import pandapower an create an empty network:

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_ext_grid,
    create_transformer,
    create_line,
    create_switch,
    create_load,
    create_sgen,
    create_gen,
    create_shunt
)

In [ ]:
net = create_empty_network() #create an empty network

### Buses

<img src="pics/example_network_simple_buses.png">

We now create the three high voltage (vn_kv=110.) and four medium voltage (vn_kv=20.) buses.

In [ ]:
bus1 = create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = create_bus(net, name="MV Main Bus", vn_kv=20, type="b")
bus6 = create_bus(net, name="MV Bus 1", vn_kv=20, type="b")
bus7 = create_bus(net, name="MV Bus 2", vn_kv=20, type="b")

Bus 3 and bus 4 are classified as nodes (type="n"), all other bus types are declared as busbars (type="b"):

In [ ]:
net.bus # show bus table

All create functions return the pandapower index of the element that was created, for example the variable bus6 is now equal to the index of the bus with the name "MV Station 2" (which is 5):

In [ ]:
bus6

We use these variables for creating bus and branch elements in the following.

### External Grid

<img src="pics/example_network_simple_ext_grid.png">

We now create an external grid connection that serves as slack node for the power flow calculation. The voltage of the external grid is set to a magnitude of 1.02 per unit and 50 degrees voltage angle:

In [ ]:
create_ext_grid(net, bus1, vm_pu=1.02, va_degree=50) # Create an external grid connection

net.ext_grid #show external grid table

### Transformer 

<img src="pics/example_network_simple_trafo.png">

The transformer connects the high-voltage with the medium-voltage side of the grid. The high-voltage bus of the transformer is connected to Bus3 and on the medium-voltage side the transformer is linked to Bus4. We select the standard type "25 MVA 110/20 kV" from the pandapower basic standard type library:

In [ ]:
trafo1 = create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

 The detailled transformer parameters, such as short circuit voltages, rated power or iron losses are automatically loaded from the standard type library (see [standard type library tutorial](std_types.ipynb)) and stored in the transformer table:

In [ ]:
net.trafo #show transformer table

### Lines

The network includes three medium voltage lines and one high voltage line. The bus connections and line lengths are defined in the network diagram:

<img src="pics/example_network_simple_lines.png">

The line parameters are once again taken from the standard type library (see [standard type library tutorial](std_types.ipynb)). We use different line lengths and standard types for each line:

In [ ]:
line1 = create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV",  name="Line 1")
line2 = create_line(net, bus5, bus6, length_km=2.0, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3 = create_line(net, bus6, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4 = create_line(net, bus7, bus5, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

The full line table looks like this:

In [ ]:
net.line # show line table

### Switches

There are two circuit breakers on the high- and low voltage side of the transformer which connect two buses (Bus2-Bus3 and Bus4-Bus5). These bus-bus switches can be defined with et="b". 
<img src="pics/example_network_simple_switches.png">

In [ ]:
sw1 = create_switch(net, bus2, bus3, et="b", type="CB", closed=True)
sw2 = create_switch(net, bus4, bus5, et="b", type="CB", closed=True)

Furthermore, we equip all bus/line connections in the medium voltage level with load break switches ("LBS") as shown in the network diagram. Bus/Line switches are defined with et="l":

In [ ]:
sw3 = create_switch(net, bus5, line2, et="l", type="LBS", closed=True)
sw4 = create_switch(net, bus6, line2, et="l", type="LBS", closed=True)
sw5 = create_switch(net, bus6, line3, et="l", type="LBS", closed=True)
sw6 = create_switch(net, bus7, line3, et="l", type="LBS", closed=False)
sw7 = create_switch(net, bus7, line4, et="l", type="LBS", closed=True)
sw8 = create_switch(net, bus5, line4, et="l", type="LBS", closed=True)

The switch table now shows all switches. The bus colum contains the index of the bus the switch is connected to. For bus switches (et="b"), the element column contains the index of the second bus the switch connects to. For line switches (et="l"), the element column contains the index of the line the switch connects to. All switches are closed.

In [ ]:
net.switch # show switch table

### Load

<img src="pics/example_network_simple_load.png">

The load element is used to model by default constant active and reactive power consumption. We create a 2 MW / 4 MVar load with a scaling factor of 0.6:

In [ ]:
create_load(net, bus7, p_mw=2, q_mvar=4, scaling=0.6, name="load")

net.load

#### Voltage dependent loads - ZIP load model
One can observe load parameters `const_z_percent` and `const_i_percent` which are set to 0 by default. They can be used in order to define part of a load which is voltage-dependent using the so-called **ZIP load** model, which allows a load definition as a composition of constant power, constant current and constant impedance.  

As an example, we create a 2 MW / 4 MVar load with 30% of constant impedance and 20% of constant current:

In [ ]:
create_load(net, bus7, p_mw=2, q_mvar=4, const_z_percent=30, const_i_percent=20, name="zip_load")

net.load

### Static Generator

<img src="pics/example_network_simple_sgen.png">

The static generator element is used to model constant active and reactive power generation. Since the [signing system](https://pandapower.readthedocs.io/en/latest/_images/p_q_reference_system.png) used in pandapower for generation units is from a generator's point of view, the active power has to be positive to model generation. We create a static generator with 2 MW generation and 500 kVar.

In [ ]:
create_sgen(net, bus7, p_mw=2, q_mvar=-0.5, name="static generator")

net.sgen

### Voltage controlled Generator

<img src="pics/example_network_simple_gen.png">

The generator element is used to model voltage controlled active power generation. We define it with a active power generation (positive) and a voltage set point:

In [ ]:
create_gen(net, bus6, p_mw=6, max_q_mvar=3, min_q_mvar=-3, vm_pu=1.03, name="generator") 

net.gen

### Shunt


<img src="pics/example_network_simple_shunt.png">

The shunt is defined by its active and reactive power consumption at rated voltage. Once again, the signing system is from a consumers point of view. We want to model a a capacitator bank, and therefore have to assign a negative reactive power to the shunt:

In [ ]:
create_shunt(net, bus3, q_mvar=-0.96, p_mw=0, name='Shunt')

net.shunt

If you want to learn how to create more complicated networks, continue with the [advanced create tutorial](create_advanced.ipynb). If you want to learn about how to run a loadflow, continue with the [power flow tutorial](powerflow.ipynb). 